# Introduction

This notebook demonstrates how to train custom openWakeWord models using pre-defined datasets and an automated process for dataset generation and training. While not guaranteed to always produce the best performing model, the methods shown in this notebook often produce baseline models with releatively strong performance.

Manual data preparation and model training (e.g., see the [training models](training_models.ipynb) notebook) remains an option for when full control over the model development process is needed.

At a high level, the automatic training process takes advantages of several techniques to try and produce a good model, including:

- Early-stopping and checkpoint averaging (similar to [stochastic weight averaging](https://arxiv.org/abs/1803.05407)) to search for the best models found during training, according to the validation data
- Variable learning rates with cosine decay and multiple cycles
- Adaptive batch construction to focus on only high-loss examples when the model begins to converge, combined with gradient accumulation to ensure that batch sizes are still large enough for stable training
- Cycical weight schedules for negative examples to help the model reduce false-positive rates

See the contents of the `train.py` file for more details.

# Environment Setup

To begin, we'll need to install the requirements for training custom models. In particular, a relatively recent version of Pytorch and custom fork of the [piper-sample-generator](https://github.com/dscripka/piper-sample-generator) library for generating synthetic examples for the custom model.

**Important Note!** Currently, automated model training is only supported on linux systems due to the requirements of the text to speech library used for synthetic sample generation (Piper). It may be possible to use Piper on Windows/Mac systems, but that has not (yet) been tested.

In [20]:
## Environment setup

# install piper-sample-generator (currently only supports linux systems)
!if [ ! -d "piper-sample-generator" ]; then git clone https://github.com/rhasspy/piper-sample-generator; fi
!wget -O piper-sample-generator/models/en_US-libritts_r-medium.pt 'https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/en_US-libritts_r-medium.pt'

# install openwakeword (full installation to support training)
!if [ ! -d "openwakeword" ]; then git clone --branch auto_training https://github.com/dscripka/openwakeword; fi
!pip install -e ./openwakeword[full]
!cd openwakeword

# Install torchcodec for audio decoding
!pip install torchcodec

--2025-12-19 11:17:20--  https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/en_US-libritts_r-medium.pt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/642029941/73f4af3c-7cf8-4547-a7b9-3bd29e7f3c33?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-19T11%3A51%3A06Z&rscd=attachment%3B+filename%3Den_US-libritts_r-medium.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-19T10%3A50%3A56Z&ske=2025-12-19T11%3A51%3A06Z&sks=b&skv=2018-11-09&sig=r1kGPjATKdC4mhTU2z9dhq4ePcP0EdAEzfziZQ7CS4M%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NjE0NjY0MCwibmJmIjoxNzY2MTQzMDQwLCJwYXRoIjoic

In [25]:
!sudo apt-get update
!sudo apt-get install -y espeak-ng build-essential python3-dev
!pip install setuptools wheel

# Install piper-phonemize from source as direct pip install failed
!if [ ! -d "piper-phonemize" ]; then git clone https://github.com/rhasspy/piper-phonemize; fi
!pip install -e ./piper-phonemize

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (1,777 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.

In [26]:
# Imports

import os
import numpy as np
import torch
import sys
from pathlib import Path
import uuid
import yaml
import datasets
import scipy
from tqdm import tqdm

# Download Data

When training new openWakeWord models using the automated procedure, four specific types of data are required:

1) Synthetic examples of the target word/phrase generated with text-to-speech models

2) Synthetic examples of adversarial words/phrases generated with text-to-speech models

3) Room impulse reponses and noise/background audio data to augment the synthetic examples and make them more realistic

4) Generic "negative" audio data that is very unlikely to contain examples of the target word/phrase in the context where the model should detect it. This data can be the original audio data, or precomputed openWakeWord features ready for model training.

5) Validation data to use for early-stopping when training the model.

For the purposes of this notebook, all five of these sources will either be generated manually or can be obtained from HuggingFace thanks to their excellent `datasets` library and extremely generous hosting policy. Also note that while only a portion of some datasets are downloaded, for the best possible performance it is recommended to download the entire dataset and keep a local copy for future training runs.

In [28]:
!pip install torchcodec

In [29]:
# Download room impulse responses collected by MIT
# https://mcdermottlab.mit.edu/Reverb/IR_Survey.html

output_dir = "./mit_rirs"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
rir_dataset = datasets.load_dataset("davidscripka/MIT_environmental_impulse_responses", split="train", streaming=True)

# Save clips to 16-bit PCM wav files
for row in tqdm(rir_dataset):
    name = row['audio']['path'].split('/')[-1]
    scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))

Resolving data files:   0%|          | 0/270 [00:00<?, ?it/s]

0it [00:00, ?it/s]


ImportError: To support decoding audio data, please install 'torchcodec'.

In [12]:
# Download room impulse responses collected by MIT
# https://mcdermottlab.mit.edu/Reverb/IR_Survey.html

output_dir = "./mit_rirs"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
rir_dataset = datasets.load_dataset("davidscripka/MIT_environmental_impulse_responses", split="train", streaming=True)

# Save clips to 16-bit PCM wav files
for row in tqdm(rir_dataset):
    name = row['audio']['path'].split('/')[-1]
    scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))

Resolving data files:   0%|          | 0/270 [00:00<?, ?it/s]

0it [00:00, ?it/s]


ImportError: To support decoding audio data, please install 'torchcodec'.

In [19]:
## Download noise and background audio

# Audioset Dataset (https://research.google.com/audioset/dataset/index.html)
# Load one part of the audioset using datasets library.
# For full-scale training, it's recommended to download the entire dataset from
# https://huggingface.co/datasets/agkphysics/AudioSet, and
# even potentially combine it with other background noise datasets (e.g., FSD50k, Freesound, etc.)

output_dir_audioset = "./audioset_16k" # Use a specific output directory for AudioSet
if not os.path.exists(output_dir_audioset):
    os.mkdir(output_dir_audioset)

# Load balanced train split from AudioSet. Adjust `n_audioset_clips_to_process` for faster execution if needed.
# This will stream the data, convert to 16kHz, and save locally.
print("Downloading and processing AudioSet (balanced train split)...")
audioset_stream = datasets.load_dataset("agkphysics/AudioSet", split="train", streaming=True)
audioset_stream = audioset_stream.cast_column("audio", datasets.Audio(sampling_rate=16000))

n_audioset_clips_to_process = 5000 # Process a reasonable number of clips for an example
processed_audioset_count = 0
for row in tqdm(audioset_stream):
    if processed_audioset_count >= n_audioset_clips_to_process:
        break
    # Generate a unique filename for each clip
    name = f"audioset_clip_{processed_audioset_count}_{uuid.uuid4().hex}.wav"
    scipy.io.wavfile.write(os.path.join(output_dir_audioset, name), 16000, (row['audio']['array']*32767).astype(np.int16))
    processed_audioset_count += 1
print(f"Finished processing {processed_audioset_count} AudioSet clips.")

# Free Music Archive dataset (https://github.com/mdeff/fma)
# This dataset currently cannot be loaded using datasets.load_dataset due to
# "Dataset scripts are no longer supported" error.
# It is commented out. Consider finding an alternative background noise dataset
# or manually downloading and processing FMA if needed.
# output_dir_fma = "./fma"
# if not os.path.exists(output_dir_fma):
#     os.mkdir(output_dir_fma)
# fma_dataset = datasets.load_dataset("rudraml/fma", name="small", split="train", streaming=True)
# fma_dataset = iter(fma_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000)))

# n_hours = 1  # use only 1 hour of clips for this example notebook, recommend increasing for full-scale training
# for i in tqdm(range(n_hours*3600//30)):  # this works because the FMA dataset is all 30 second clips
#     row = next(fma_dataset)
#     name = row['audio']['path'].split('/')[-1].replace(".mp3", ".wav")
#     scipy.io.wavfile.write(os.path.join(output_dir_fma, name), 16000, (row['audio']['array']*32767).astype(np.int16))
#     i += 1
#     if i == n_hours*3600//30:
#         break


Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

0it [00:10, ?it/s]


ImportError: To support decoding audio data, please install 'torchcodec'.

In [ ]:
# Download pre-computed openWakeWord features for training and validation

# training set (~2,000 hours from the ACAV100M Dataset)
# See https://huggingface.co/datasets/davidscripka/openwakeword_features for more information
!wget https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/openwakeword_features_ACAV100M_2000_hrs_16bit.npy

# validation set for false positive rate estimation (~11 hours)
!wget https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/validation_set_features.npy

# Define Training Configuration

For automated model training openWakeWord uses a specially designed training script and a [YAML](https://yaml.org/) configuration file that defines all of the information required for training a new wake word/phrase detection model.

It is strongly recommended that you review [the example config file](../examples/custom_model.yml), as each value is fully documented there. For the purposes of this notebook, we'll read in the YAML file to modify certain configuration parameters before saving a new YAML file for training our example model. Specifically:

- We'll train a detection model for the phrase "hey sebastian"
- We'll only generate 5,000 positive and negative examples (to save on time for this example)
- We'll only generate 1,000 validation positive and negative examples for early stopping (again to save time)
- The model will only be trained for 10,000 steps (larger datasets will benefit from longer training)
- We'll reduce the target metrics to account for the small dataset size and limited training.

On the topic of target metrics, there are *not* specific guidelines about what these metrics should be in practice, and you will need to conduct testing in your target deployment environment to establish good thresholds. However, from very limited testing the default values in the config file (accuracy >= 0.7, recall >= 0.5, false-positive rate <= 0.2 per hour) seem to produce models with reasonable performance.


In [15]:
# Load default YAML config file for training
config = yaml.load(open("openwakeword/examples/custom_model.yml", 'r').read(), yaml.Loader)
config

{'model_name': 'my_model',
 'target_phrase': ['hey jarvis'],
 'custom_negative_phrases': [],
 'n_samples': 10000,
 'n_samples_val': 2000,
 'tts_batch_size': 50,
 'augmentation_batch_size': 16,
 'piper_sample_generator_path': './piper-sample-generator',
 'output_dir': './my_custom_model',
 'rir_paths': ['./mit_rirs'],
 'background_paths': ['./background_clips'],
 'background_paths_duplication_rate': [1],
 'false_positive_validation_data_path': './validation_set_features.npy',
 'augmentation_rounds': 1,
 'feature_data_files': {'ACAV100M_sample': './openwakeword_features_ACAV100M_2000_hrs_16bit.npy'},
 'batch_n_per_class': {'ACAV100M_sample': 1024,
  'adversarial_negative': 50,
  'positive': 50},
 'model_type': 'dnn',
 'layer_size': 32,
 'steps': 50000,
 'max_negative_weight': 1500,
 'target_false_positives_per_hour': 0.2}

In [17]:
# Load default YAML config file for training
# config = yaml.load(open("openwakeword/examples/custom_model.yml", 'r').read(), yaml.Loader)

# --- 核心修改部分 ---

# 1. 设置唤醒词：混合汉字和拼音，确保 TTS (语音合成) 能读对
config["target_phrase"] = ["王八蛋", "wang ba dan"]

# 2. 模型文件名：最好用英文，不要有空格
config["model_name"] = "wangbadan"

# 3. 样本数量：原文是 1000，太少了模型会很笨。
# 建议改成 5000 或 10000。5000 大约需要跑 15-20 分钟，效果勉强可用。
config["n_samples"] = 5000

# 验证集数量：保持 1000 即可，用于检测模型是否学歪了
config["n_samples_val"] = 1000

# 训练步数：配合样本量适当增加，保证学得透彻一点
config["steps"] = 10000

# --- 下面这些参数保持默认或根据之前步骤的文件路径调整 ---
# 注意：确保你的 Colab 左侧文件栏里确实有 validation_set_features.npy 等文件
# 如果是自动脚本，通常它会自动下载，不用改下面这些
config["target_accuracy"] = 0.6
config["target_recall"] = 0.25
config["background_paths"] = ['./audioset_16k', './fma']
config["false_positive_validation_data_path"] = "validation_set_features.npy"
config["feature_data_files"] = {"ACAV100M_sample": "openwakeword_features_ACAV100M_2000_hrs_16bit.npy"}

# 保存配置文件
with open('my_model.yaml', 'w') as file:
    documents = yaml.dump(config, file)

# Train the Model

With the data downloaded and training configuration set, we can now start training the model. We'll do this in parts to better illustrate the sequence, but you can also execute every step at once for a fully automated process.

In [ ]:
# Step 1: Generate synthetic clips
# For the number of clips we are using, this should take ~10 minutes on a free Google Colab instance with a T4 GPU
# If generation fails, you can simply run this command again as it will continue generating until the
# number of files meets the targets specified in the config file

!{sys.executable} openwakeword/openwakeword/train.py --training_config my_model.yaml --generate_clips

In [ ]:
# Step 2: Augment the generated clips

!{sys.executable} openwakeword/openwakeword/train.py --training_config my_model.yaml --augment_clips

In [ ]:
# Step 3: Train model

!{sys.executable} openwakeword/openwakeword/train.py --training_config my_model.yaml --train_model

After the model finishes training, the auto training script will automatically convert it to ONNX and tflite versions, saving them as `<model_name>.onnx/tflite` in the present working directory, where `<model_name>` is defined in the YAML training config file. Either version can be used as normal with `openwakeword`. I recommend testing them with the [`detect_from_microphone.py`](https://github.com/dscripka/openWakeWord/blob/main/examples/detect_from_microphone.py) example script to see how the model performs!